# Detección de Anomalías

Una **anomalía** es nada más que un punto que presenta desviaciones respecto a lo que es normal, regular o natural. 
En el ámbito de la ciberseguridad, un punto anómalo en nuestro conjunto de datos (logs de usuarios, tráfico de red, emails...) tenderá a representar un **ataque**, ya que indica un comportamiento distinto al que suele haber en nuestros sistemas. Pero, incluso en el caso de que no fuese un ataque o actividad maliciosa, querremos poder detectar conductas aómalas en nuestra organización.

## Exploración de datos

### Framework

Vamos a usar pandas para la manipulación de nuestro dataset, y matplotlib, sns y plotly para las visualizaciones de gráficos que podamos necesitar.

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

sns.set_style('white')
%matplotlib inline

### Dataset

Nuestro conjunto de datos son registros de transacciones de tarjetas de crédito de distintos usuarios, y nuestro objetivo será detectar las transacciones fraudulentas.

El dataset contiene datos reales, pero tiene la particularidad de que sus variables fueron generadas PCA (Principal Component Analysis), una técnica de reducción de dimensiones de datos.
Este tipo de métodos se aplican a los datos por varios motivos:

- Anonimización. Esta fue la motivación principal de los creadores del dataset. Tras aplicar PCA sobre las variables originales, se obtienen variables nuevas que no nos permiten deducir la información original del conjunto de datos. El inconveniente principal de usar reducción de dimensionalidad para anonimización es la pérdida de interpretabilidad de las variables, pero podemos seguir usando los datos para clasificación, detección de anomalías, etc, mientras mantengamos intacta la variable que indica la clase/etiqueta de cada dato.

- Solución del "curse of dimensionality". El coste computacional y la dificultad de analizar datos con un montón de variables, a lo que se le llama dimensionalidad alta, es muy elevado. Muchas veces se aplican técnicas como PCA o TSNE para reducir el número de dimensiones/variables de los datos antes de entrenar algoritmos de ML. Estos algoritmos se aplican para eliminar o combinar las variables que tengan correlaciones altas entre sí. De esta forma, pasamos a tener menos columnas en nuestro dataset final, lo cual facilita el entrenamiento de modelos de IA. 

Además, el uso de nombres de variables abstractos como V1, V2, V3... nos permite reducir nuestro bias hacia los datos, y aumentar así la generalización del modelo. Manteniéndonos agnósticos al dominio de aplicación, nos centramos en la selección y procesado objetivo de los datos para obtener los mejores resultados.

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

In [ ]:
df = pd.read_csv('data/creditcard.csv', index_col=0)

In [ ]:
df.head(10)

Todas las variables son números decimales, ahorrándonos el problema de tener que lidiar con variables categóricas. La única excepción es 'Class', la etiqueta que distingue transacciones normales de las fraudulentas, que es un valor entero (0 o 1).

In [ ]:
df.dtypes

In [ ]:
df.describe()

En dataset no nos falta ningún valor, si se diese el caso tendríamos que rellenar el campo (usando, por ejemplo, la media) o eliminar la fila correspondiente.

In [ ]:
df.isna().values.any()

Tenemos un dataset no balanceado, donde la clase 1 (fraude) tiene muchas menos instancias que la clase 0 (datos normales).

In [ ]:
df['Class'].value_counts()

Podemos generar un report 

In [ ]:
prof = ProfileReport(df)
prof.to_file(output_file='reports/creditcard_report.html')


#### Correlación entre variables

Antes de aplicar ningún tipo de AI a nuestros datos, podemos representar su correlación lineal para ver si hay algún tipo de influencia entre sus valores.

En este caso concreto, veremos que las columnas que empiezan por "V" no están relacionadas en absoluto. Esto es un resultado directo de haber sido generadas con PCA, donde queremos poder representar nuestros datos con el mínimo número de dimensiones, por lo que eliminaremos/agruparemos variables que estén relacionadas.

La variable Amount no ha sido modificada, por lo que sí tenemos información sobre ella. Por ejemplo, vemos que tiene una correlación negativa bastante fuerte con V2.

In [ ]:
fig = px.imshow(df.corr(),
    labels=dict(color='Correlación'),
    text_auto=True,
    color_continuous_scale='tropic',
    color_continuous_midpoint=0
)
fig.update_layout(
    title_text='Correlación lineal entre variables',
    paper_bgcolor='white',
    height=500, width=800
)

Lo que más nos interesa aquí es ver la correlación entre las variables y la clase (Class). Vemos que no todas influyen de la misma forma, y hay algunas variables que parece que no afectan a la clase en absoluto (de forma lineal).

#### Dispersión a pares de las variables

Aparte de las correlaciones, podemos representar cómo se distribuyen nuestros puntos para cada par de variables (lo llamamos pairplot).

En primer lugar, para reducir el tiempo de cálculo y tener una visualización más clara, creamos un dataframe nuevo con todos los datos anómalas y 3000 muestras de los datos normales.

In [ ]:
normal = df[df['Class'] == 0].sample(3000)
anomaly = df[df['Class'] == 1]
sampled_df = pd.concat([normal, anomaly], ignore_index=True)
sampled_df['Class'].value_counts()

Para empezar, seleccionamos las variables que tengan un nivel de correlación relativamente alto con la columna Class. Por ejemplo, a partir de 0.1.

In [ ]:
min_correlation = 0.10
high_corr_cols = list(df.corr()['Class'][abs(df.corr()['Class']) > min_correlation].index)
print(f'Variables con correlación > {min_correlation} o < -{min_correlation}: {high_corr_cols}')

El gráfico representa la distribución de nuestros puntos para ambas clases (Rojo = Normal, Azul Claro = Fraude), para cada par de variables.
En la diagonal tenemos histogramas de cada clase para la misma variable. Nos interesa ver cómo de separables son las dos clases para estos conjuntos de variables.

Podemos apreciar que, dependiendo del conjunto de variables, hay más o menos superposición entre los puntos. Si pudiesemos separar completamente las dos clases para alguna variable o conjunto de variables, tendríamos un problema de clasificación muy sencillo.

In [ ]:
g = sns.pairplot(sampled_df[high_corr_cols], 
             hue="Class", 
             palette='hls',
             plot_kws={"alpha": 0.2})
g.fig.set_dpi(200)

¿Qué pasa si comparamos el gráfico anterior al que obtenemos representando variables con muy poca correlación (< 0.05)?

Escoged algunas variables y representémoslas en un nuevo pairplot:

In [ ]:
low_corr_cols = [] #escoged columnas!
g = sns.pairplot(sampled_df[low_corr_cols + ['Class']], 
             hue='Class', 
             palette='hls',
             plot_kws={'alpha': 0.2})
g.fig.set_dpi(200)

¿Qué deberíamos observar en estos gráficos, para las variables de correlación baja?

Tanto los grupos de puntos como los histogramas de estas clases se solapan mucho más que en los gráficos del pairplot de arriba (¡algunas son prácticamente indistinguibles!).
A priori, esto **podría** indicar que estas variables no le aportarán información valiosa al modelo para distinguir entre la clase normal y anómala.

## Detección de anomalías & entrenamiento no supervisado

Vamos a centrarnos en técnicas de entrenamiento **no supervisado** para la detección de anomalías. ¿Qué quiere decir "no supervisado"? Significa que NO necesitamos etiquetas en nuestro conjunto de datos para su entrenamiento. No parece para tanto, pero es una cualidad extremadamente útil para el tipo de situaciones (¡especialmente en ciberseguridad!) donde queremos aplicar detección de anomalías, porque tiene las siguientes implicaciones:

- No tenemos que etiquetar nuestros datos. Normalmente, nosotros no sabemos de antemano cuáles de nuestros correos o logs son maliciosos **antes** de aplicar técnicas de análisis de datos. Es muy dificil etiquetar correctamente este tipo de información, y la mayor parte de organizacione no la tiene disponible. En general, en aplicaciones de ciberseguridad, es muy poco probable que encontremos un dataset etiquetado que se adapte bien y sirva para nuestros objetivos.

- Nuestro modelo de aprendizaje no supervisado va a poder detectar **todo tipo** de comportamientos maliciosos, mientras sean anómalos. Incluso si tuviesemos a mano un dataset perfectamente etiquetado, si utilizamos algorimos supervisados es probable que aprenda a detectar muy bien los ataques que aparecen en el dataset, pero esto no tiene por qué extenderse a otros tipos de ataques que puedan suceder en el futuro.

### Isolation Forest

El Isolation Forest es el modelo de ML que vamos a usar como método de detección de anomalías. Como indica su nombre, es un algoritmo basado en árboles de decisión.
Intenta separar o aislar las instancias anómalas a base de hacer particiones sobre nuestro conjunto de datos. En cada árbol de decisión, va seleccionando valores de división aleatorios para cada variable, y divide los datos en dos grupos dependiendo de si tienen valores mayores o menores que el valor de división. Cada grupo crea una rama nueva en el árbol, donde se realizará la siguiente partición. Esto se repite hasta que todos los datos queden aislados, o hasta que se llegue a un nivel de profundidad máximo.

Finalmente, la puntuación de anomalía de cada dato se decide en función de la profundidad a la que se encuentra su rama. Los datos anómalos serán más sencillos de separar del resto que los datos normales y necesitarán un número menor de particiones, por tanto se encontrarán en ramas muy poco profundas del árbol. La puntuación o score final dependerá de los valores que se obtengan en todos los árboles de decisión para ese dato.

Para separar los datos normales de los anómalos, hace falta definir un threshold o umbral de decisión. Por defecto, el Isolation Forest tiene un umbral de decisión de 0.5, por tanto los datos que obtengan puntuaciones mayores a 0.5 se clasifican como anomalías.

En la imagen podemos ver las particiones de distintos árboles del Isolation Forest, con las anomalías representadas en rojo.

<img src="images/iforest.jpg" width="1000"/>


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler, RobustScaler

### División de los datos

Para entrenar y evaluar nuestro modelo, primero dividimos el dataset en los datos que usamos para entrenar (X_train) y los datos que usamos para predecir (X_test).

In [ ]:
#df = df[high_corr_cols]

### Entrenamiento semi-supervisado

Es muy similar al no supervisado, seguimos sin necesitar etiquetas para entrenar el modelo pero asumimos que sólo hay datos normales en nuestro set de entrenamiento.

In [ ]:
normal = df[df['Class'] != 1].drop('Class', axis=1)
anomalies = df[df['Class'] == 1].drop('Class', axis=1)

In [ ]:
X_train, X_test = train_test_split(normal, random_state=22, test_size=0.01) #cambiar test_size para tener más o menos % de datos en X_test
y_test = pd.Series([0]*X_test.shape[0])
X_test = X_test.append(anomalies)
y_test = y_test.append(pd.Series([1]*anomalies.shape[0]))
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(f'Tamaño de datos de entrenamiento: {len(X_train)}')
print(f'Tamaño de datos de evaluación: {len(X_test)} ({len(anomalies)} anomalías)')

### Entrenamiento no supervisado

No necesitamos etiquetas en nuestro set de entrenamiento, sólo en el de test para evaluar nuestro modelo.

In [ ]:
X_train, X_test = train_test_split(normal, random_state=22, test_size=0.01) #cambiar test_size para tener más o menos % de datos en X_test
train_anomalies, test_anomalies = train_test_split(anomalies, random_state=22, test_size=0.9)
y_test = pd.Series([0]*X_test.shape[0])
X_train = X_train.append(train_anomalies)
X_test = X_test.append(test_anomalies)
y_test = y_test.append(pd.Series([1]*test_anomalies.shape[0]))
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(f'Tamaño de datos de entrenamiento: {len(X_train)} ({len(train_anomalies)} anomalías)')
print(f'Tamaño de datos de evaluación: {len(X_test)} ({len(test_anomalies)} anomalías)')

In [ ]:
anomaly_pctg = len(train_anomalies) / len(X_train)
anomaly_pctg

### Entrenamiento

#### Parámetros del Isolation Forest

- n_estimators: Número de árboles.
- max_samples: Número de muestras con las que se entrenará cada árbol.
- contamination: Proporcion esperada de anomalías en el conjunto de datos. Sirve para determinar el umbral de decisión.

In [ ]:
params = {
    'n_estimators': 50, #200, 250...
    'max_samples': 50, #200, 250...
    'contamination': 'auto', # 'auto', 0.1, 0.001 ...
}

pipeline = Pipeline(
    [   
        ('robust_scaler', RobustScaler()),
        ('norm', MinMaxScaler(feature_range=(0, 1))),
    ]
)

isolation_model = IsolationForest(
    **params,
    warm_start=True,
    verbose=1,
    n_jobs=4, #para lanzar hilos concurrentes
    random_state=42
)

isolation_model.fit(pipeline.fit_transform(X_train))

In [ ]:
model_thresh = - isolation_model.offset_
print(f'Umbral de decisión: {model_thresh}')

### Predicción

Tenemos dos funciones importantes:

- score_samples: la función que nos dice la puntuación de anomalía de cada dato

- predict: la función que decide si un dato es anómalo o no. Nuestra implementación del isolation forest devuelve 1 para datos normales y -1 para datos anómalos, así que los cambiamos por 0 y 1 respectivamente para que se adapte a nuestras etiquetas en la columna 'Class'.

Tanto las predicciones (pred) como las puntuaciones (scores) son muy importantes para evaluar correctamente el modelo.

In [ ]:
scores = isolation_model.score_samples(pipeline.transform(X_test))
y_pred = X_test.copy()
y_pred['pred'] = isolation_model.predict(pipeline.transform(X_test))
y_pred['pred'].replace({1: 0, -1: 1}, inplace=True) 
y_pred['scores'] = [-score for score in scores]
y_pred['Class'] = y_test

In [ ]:
y_pred[['Class','pred','scores']].sample(200).head()

### Evaluación

In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
)
import plotly.express as px
from plotly import graph_objects as go
import plotly.figure_factory as ff

#### Matriz de confusión & métricas derivadas

La matriz de confusión nos permite ver el desempeño de nuestro modelo, representando las instancias en las que acertó y falló para cada clase.

Las filas de la matriz representan la clase real, y las columnas son la predicción de nuestro modelo. Como tenemos sólo dos clases, nuestra matriz tendrá una dimensión de 2x2, de la siguiente forma: 

<img src="images/cm.png" width="600"/>

En la diagonal principal están todas las veces que el modelo acertó en sus predicciones para cada clase (Verdaderos Negativos - TN, y Verdaderos Positivos - VN), aquí es donde queremos que esté la mayor cantidad de instancias posibles.
En la esquina superior derecha tenemos los Falsos Positivos (FP), es decir, las veces que el modelo dijo que algo era una anomalía cuando realmente era un dato normal.
En la esquina inferior izquierda tenemos lo opuesto, las veces que el algoritmo predijo que una transacción era normal, cuando realmente era fraude. Estos son los Falsos Negativos (FN).

In [ ]:
cm = confusion_matrix(y_true=y_pred['Class'], y_pred=y_pred['pred'])
fig, ax = plt.subplots(figsize=[4, 4])
sns.heatmap(cm, ax=ax, annot=True, fmt='g', cmap='Purples')
ax.set_ylabel('Clase real')
ax.set_xlabel('Predicción')

A partir de la matriz de confusión podemos sacar varias métricas importantes.

- Accuracy: El porcentaje total de elementos que clasificamos correctamente. Fórmula: (TP + TN) / (TP + TN + FP + FN)

- Precision: Para la clase positiva, permite medir la calidad de las clasificaciones del modelo. Responde a: ¿entre todos los datos que el modelo dice que son anómalos, cuales realmente lo son? Fórmula = TP / (TP + FP)

- Recall: Para la clase positiva, permite medir la cantidad de veces que el modelo acierta. Responde a: ¿entre todos los datos anómalos, cuántos somos capaces de detectar? Fórmula = TP / (TP + FN)

- F1: Media armónica entre precision y recall. Fórmula = (2 * precision * recall) / (precision + recall)

In [ ]:
tn, fp, fn, tp = cm.ravel()
metrics = {}
metrics['recall'] = tp / (tp + fn)  
metrics['accuracy'] = (tp + tn) / (tp + tn + fp + fn)
metrics['precision'] = tp / (tp + fp)
metrics['f1'] = 2 * (
    (metrics['precision'] * metrics['recall'])
    / (metrics['precision'] + metrics['recall'])
)
pd.DataFrame.from_records(metrics, index=[1])

Vemos que la accuracy nos da, por lo general, valores superiores al resto de métricas. Esto se debe a que tenemos un dataset no balanceado, donde la clase normal tiene muchos más puntos que la anómala. La accuracy no distingue entre clases, sólo si el modelo acierta o no, por lo que se ve muy fuertemente influenciada por la clase de mayor tamaño. Por lo general, preferimos usar el precision y recall como métricas cuando estamos trabajando con datasets no balanceados.

#### Score Distribution

Las métricas que hemos visto hasta ahora evalúan el modelo según sus aciertos, porque nos basamos en las etiquetas de clasificación finales: 0 o 1.

Sin embargo, también podemos querer visualizar las puntuaciones de anomalía que obtienen nuestros datos, para ver cómo de bien distingue el modelo entre las dos clases, y si es capaz de separarlas.

Si representamos un histograma de las puntaciones que obtiene cada clase comparadas con el umbral de clasificación de nuestro modelo (recordamos que por defecto es 0.5), obtenemos el siguiente gráfico:

In [ ]:
thresh = -isolation_model.offset_
hist_data = [
    y_pred['scores'][y_pred['Class'] == 0],
    y_pred['scores'][y_pred['Class'] == 1],
]
colors = ['#835AF1', '#7FA6EE']

fig = ff.create_distplot(
    hist_data,
    ['Normal', 'Anomalía'], 
    bin_size=.015, # cambiar valor para ver con más o menos detalle la distribución de los datos
    histnorm='probability density',
    colors=colors
)
fig.add_vline(
    x=thresh, 
    line=dict(dash='dot'),
    annotation_text=f'  umbral de decisión={thresh:.4f}',
    annotation=dict(font_size=9),
    fillcolor='green'
)

fig.update_layout(height=500, width=800, title_text='Distribución de scores')
fig.update_xaxes(title_text='Score')
fig.show()

Si comparamos este histograma con los que habíamos obtenido al principio para las variables originales del dataset, podemos ver claras diferencias. Como mínimo, ahora no hay solape entre las partes más densas de cada curva. Podemos trazar una linea que divide ambas clases.

Esencialmente, lo que consigue nuestro modelo de detección de anomalías es elaborar una función para definir una variable nueva a partir de nuestro conjunto de datos, la score, que separa lo mejor posible los valores normales de las anomalías.
Si nuestro modelo fuese perfecto, las curvas estarían perfectamente divididas por el umbral de decisión. 

Podemos ver que si desplazamos nuestro umbral la matriz de confusión y las métricas obtenidas cambiarían. Para ver estos cambios en los gráficos, podemos volver a entrenar el algoritmo cambiando el parámetro 'contamination' a valores no automáticos, haciendo que cambie su threshold en función de las puntuaciones de los datos de entrenamiento.

**Nota: este tipo de gráfico normaliza las curvas para que tengan dimensiones similares y se visualicen mejor sus tendencias. En realidad, tenemos muchas menos anomalías que datos normales, lo cual resultaría en un histograma con valores más pequeños.

### AUC ROC

La AUC ROC es la forma típica de cuantificar esta capacidad del modelo de separar correctamente las dos clases. El AUC es el valor del área bajo la curva ROC, que es una curva de **probabilidad**. 
Para obtenerla, vamos desplazando nuestro umbral de decisión (threshold), y observamos cómo evolucionan nuestra TPR (Tasa de Verdaderos Positivos) y FPR (Tasa de Falsos Positivos). Para obtener una AUC ideal, nuestra curva ROC pasaría por el punto [0,1], lo cual querría decir que existe un umbral de decisión que nos permite tener un 0% de falsos positivos a la vez que un 100% de verdaderos positivos.

El mejor valor posible de AUC es 1, pero el peor valor no es 0 sino 0.5, porque quiere decir que el modelo tiene una probabilidad de 0.5 de acertar, la misma que si escojemos las clases al azar. Si la AUC da 0 es que el modelo está etiquetando las clases al revés (¡pero esto sigue queriendo decir que puede distinguir entre ellas!).

In [ ]:
labels=y_test
scores=y_pred['scores']

fpr, tpr, _ = roc_curve(labels, scores)
thr = sum(labels) / len(labels)

fig = px.area(
    x=fpr, 
    y=tpr,
    labels=dict(x='FPR',y='TPR'),
    title=f'Curva ROC (AUC={auc(fpr, tpr):.4f})',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.add_trace(
    go.Scatter(
    x=[0,1], y=[0,1], 
    line=dict(dash='dot', color='purple'), 
    name='AUC-ROC base = 0.5')
)

fig.update_layout(height=400, width=600)
fig.show()

Nuestra AUC nos indica que el modelo tiene una capacidad muy alta para separar entre ambas clases, independientemente del umbral de decisión que seleccionemos.

En la mayor parte de modelos de ML no obtendremos una AUC perfecta, por lo que buscaremos encontrar un compromiso entre los verdaderos y falsos positivos. Depende de nosotros lo que queramos priorizar a la hora de definir las etiquetas: detectar todos los fraudes posibles a cambio de tener falsos positivos, lo cual podría tener consecuencias negativas para los usuarios legítimos de nuestras aplicaciones, o intentar reducir los falsos positivos detectando menos anomalías.

En este caso, el impacto de no detectar un fraude en una transacción bancaria es mucho más alto que el de actuar sobre un falso positivo. Generalmente preferimos que el banco bloquee la transacción, nos envíe un SMS de alerta o nos llame, incluso si era una actividad válida, antes que dejar que alguien use nuestra tarjeta de crédito de forma fraudulenta.